In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

## Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event_data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    #print('root: ', root, '\ndirs: ', dirs, '\nfiles: ', files)
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(filepath,'*'))
    print(file_path_list)

## Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list: 
    # read the csv file in the current filepath
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)        
        # for each data row in the csv file, append it      
        for line in csvreader:
            full_data_rows_list.append(line) 
print("Total Number of Rows: ", len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

## Creating Apache Cassandra cluster

In [ ]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

In [ ]:
# create Keyspace

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [ ]:
# set keyspace

try:
    session.set_keyspace('udacity')
except exception as e:
    print(e)

## Creating queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## 1.

Because the question is asking for the name of the artist, the song's title, and the song's length, it is apparent that our data model will have to include the **artist_title**, **song_title**, and **song_length** columns from **event_datafile_new.csv**. In our query, we will use the `SELECT` statement to retreive these values.

Because the question is asking to search for these values based on a specified **sessionId** and **itemInSession**, we will need to include these colums from **event_datafile_new.csv** in our data model as well, and we will use these columns to partition the data. In our query, will use the `WHERE` clause to specify that we want to search on these columns, which will be our partition.

We will call the data model, or table, *music_history*.

With all this in mind, we get the following query:

`SELECT artist_name, song_title, song_length FROM music_history WHERE sessionID=338 AND itemInSession=4`

To create our table that is rightly partioned by the **sessionId** and **itemInSession** columns, we will set our `PRIMARY KEY` to these columns by using **sessionId** as our `PARTITION KEY` and **itemInSession** as our `CLUSTERING KEY`:

`CREATE TABLE IF NOT EXISTS music_history
(
    sessionId int,
    itemInSession int,
    artist_name text, 
    song_title text, 
    song_length float, 
    PRIMARY KEY (sessionId, itemInSession)
)`

In [ ]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS music_history "
query = query + "(sessionId int, itemInSession int, artist_name text, song_title text, song_length float, PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# Populating music_history table with data from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_history (sessionId, itemInSession, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [ ]:
# Executing Query 1

query = "SELECT artist_name, song_title, song_length FROM music_history WHERE sessionID=139 AND itemInSession=1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
print(pd.DataFrame(list(rows)))

## 2.

Because the question is asking for the name of the artist, the song's title, and the user's full name, it is apparent that our data model will have to include the **artist_title**, **song_title**, **firstName** and **lastName** columns from **event_datafile_new.csv**. In our query, we will use the `SELECT` statement to retreive these values.

Because the question is asking to search for these values based on a specified **userId** and **sessionId**, we will need to include these colums from **event_datafile_new.csv** in our data model as well, and we will use these columns to partition the data. In addition, the question asks to sort the songs by **itemInSession**. To achieve that, we will need to use **itemInSession** to partition our data (will explain why later), but these are not needed in our `SELECT` query. In our query, will use the `WHERE` clause to specify that we want to search only on the **userId** and **sessionId** columns of our partition.

We will call the data model, or table, *user_history*.

With all this in mind, we get the following query:

`SELECT artist_name, song_title, firstName, lastName FROM user_history WHERE userId=10 AND sessionId=182`

To create our table that is rightly partioned by the **userId**, **sessionId**, and **itemInSession** columns, we will do a couple things to create our `PRIMARY KEY`:
- Group **userId** and **sessionId** together as our `PARTITION KEY`
- Set **itemInSession** as our `CLUSTERING KEY`


`CREATE TABLE IF NOT EXISTS music_history
(
    userId int, 
    sessionID int, 
    itemInSession int, 
    song_title text, 
    artist_name text,
    firstName text, 
    lastName text, 
    PRIMARY KEY ((userId, sessionId), itemInSession)
)`

The reason we need **itemInSession** included in our partition, is because Apache Cassandra automaically groups data by the order they are listed in the `PRIMARY KEY` and sorts them in ascending order. 

In [ ]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS user_history "
query = query + "(userId int, sessionID int, itemInSession int, song_title text, artist_name text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# Populating music_history table with data from event_datafile_new.csv

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_history (userId, sessionID, itemInSession, song_title, artist_name, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[9], line[0], line[1], line[4]))

In [ ]:
# Executing Query 2

query = "SELECT artist_name, song_title, firstName, lastName FROM user_history WHERE userId=8 AND sessionId=139"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

print(pd.DataFrame(list(rows)))

## 3.

Because the question is asking for full names of users, it is apparent that our data model will have to include the **firstName** and **lastName** columns from **event_datafile_new.csv**. In our query, we will use the `SELECT` statement to retreive these values.

Because the question is asking to search for these values based on a specified song title, we will need to include the **song_title** column from **event_datafile_new.csv** in our data model as well, and we will use this column to partition the data. However, the song title alone won't be sufficient for giving each row of a data unique `PRIMARY KEY`, as several users can listen to the same song. So, to make sure each row has a unique `PRIMARY KEY`, we will need to include the **userId** column as well, but it will not be needed for our query. We will only search on the **song_title** column of our partition using the `WHERE` clause.

We will call the data model, or table, *song_playlist_session*.

With all this in mind, we get the following query:

`SELECT firstName, lastName FROM song_playlist_session WHERE song_title='All Hands Against His Own'`

To create our table that is rightly partioned by the **song_title** and **userId** columns, we will do a couple things to create our `PRIMARY KEY`:
- Set **song_title** as our `PARTITION KEY`
- Set **userId** as our `CLUSTERING KEY`

`CREATE TABLE IF NOT EXISTS song_playlist_session
(
    song_title text,
    userId,
    firstName text, 
    lastName text, 
    PRIMARY KEY (song_title, userId)
)`

In [ ]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(song_title text, userId int, firstName text, lastName text, PRIMARY KEY (song_title, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# Populating music_history table with data from event_datafile_new.csv

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (song_title, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [ ]:
## Executing Query 3
query = "SELECT firstName, lastName FROM song_playlist_session WHERE song_title='Pump It'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
print(pd.DataFrame(list(rows)))

### Drop the tables before closing out the sessions

In [ ]:
try:
    session.execute('DROP TABLE IF EXISTS music_history')
except Exception as e:
    print(e)
    
try:
    session.execute('DROP TABLE IF EXISTS user_history')
except Exception as e:
    print(e)
    
try:
    session.execute('DROP TABLE IF EXISTS song_playlist_session')
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()